# Party donations via SMS

This script is a modification of [my party donation scraper](https://github.com/marcelpauly/parteispenden/blob/master/parteispenden.ipynb) that feeds my Twitterbot [@grossspenden](https://twitter.com/grossspenden). It simplifies the tracking of the German parliament's website and informs me via SMS _if_ there were changes on the [list of party donations](https://www.bundestag.de/parlament/praesidium/parteienfinanzierung/fundstellen50000/2017/2017-inhalt/488236) (not _which_ changes like the Twitterbot does).

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from datetime import datetime
import time
from twilio.rest import Client

In [2]:
# read the donation information from the website

url = 'https://www.bundestag.de/parlament/praesidium/parteienfinanzierung/fundstellen50000/2017/2017-inhalt/488236'
response = requests.get(url)
doc = BeautifulSoup(response.text, 'html.parser')

rows = [row for row in doc.tbody.find_all('tr') if row.td.has_attr('colspan') == False]
donations = []

for donation in rows:
    if donation.find_all('td')[4].find('br'):
        date_of_notification = donation.find_all('td')[4].br.previousSibling
    else:
        date_of_notification = donation.find_all('td')[4].text.strip()
    if donation.find_all('td')[2].find('br'):
        donor_name = donation.find_all('td')[2].br.previousSibling.strip()
        lines = [line.nextSibling for line in donation.find_all('td')[2].find_all('br') if line.nextSibling]
        donor_address = ', '.join(lines)
    else:
        donor_name = donation.find_all('td')[2].text.strip()
        donor_address = np.nan
    donations.append({
        'party': donation.find_all('td')[0].text.strip(),
        'amount': float(donation.find_all('td')[1].text.strip().replace('.', '').replace(',', '.')),
        'donor_name': re.sub('^(Herr |Frau |Firma )', '', donor_name),
        'donor_address': donor_address,
        'date_of_receipt': datetime.strptime(donation.find_all('td')[3].text.strip(), '%d.%m.%Y'),
        'date_of_notification': datetime.strptime(date_of_notification, '%d.%m.%Y')
    })

df = pd.DataFrame(donations)

df

,amount,date_of_notification,date_of_receipt,donor_address,donor_name,party
0,68000.00,2017-07-14,2017-07-12,"Lutterstraße 14, 33617 Bielefeld",Dr. August Oetker KG,CDU
1,55000.00,2017-07-06,2017-07-05,"Zugspitzstraße 1, 2049 Pullach",Sixt GmbH & Co. Autovermietung KG,FDP
2,90000.00,2017-07-04,2017-07-03,"Uerdinger Straße 58-62, 40474 Düsseldorf",Metall NRW - Verband der Metall- und Elektro-I...,FDP
3,110000.00,2017-07-05,2017-06-30,"Uerdinger Straße 58-62, 40474 Düsseldorf",Metall NRW - Verband der Metall- und Elektro-I...,CDU
4,50001.00,2017-06-29,2017-06-26,"Seedammweg 55, 61352 Bad Homburg",Susanne Klatten,CDU
5,50001.00,2017-06-29,2017-06-26,"Seedammweg 55, 61352 Bad Homburg",Stefan Quandt,CDU
6,50001.00,2017-06-26,2017-06-23,"Seedammweg 55, 61352 Bad Homburg",Susanne Klatten,FDP
7,50001.00,2017-06-26,2017-06-23,"Seedammweg 55, 61352 Bad Homburg",Stefan Quandt,FDP
8,150000.00,2017-06-23,2017-06-22,"Lindenallee 78, 50968 Köln",Fa. R+W Industriebeteiligungen GmbH,FDP
9,70000.00,2017-06-21,2017-06-20,"Merckstraße 40, 64342 Seeheim-Jugenheim",Prof. Dr. Hans-Joachim Langmann,CDU


In [3]:
# compare the current list with the previous one and send a SMS if they differ

df_old = pd.read_csv('donations_latest.csv', parse_dates=['date_of_notification', 'date_of_receipt'])

if df.equals(df_old) == False:
    account_sid = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
    auth_token  = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
    client = Client(account_sid, auth_token)
    message = client.messages.create(
        to = '+1XXXXXXXXXX',
        from_ = '+1XXXXXXXXXX',
        body = 'ALERT! There have been changes on the parliament\'s party donation page recently: http://j.mp/2uLkWSi'
    )

In [4]:
# save the new list

datestring = time.strftime('%Y-%m-%d_%H-%M')

df.to_csv('donations_latest.csv', index=False)
df.to_csv('donations_' + datestring + '.csv', index=False)